In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os

base_path = "/content/drive/MyDrive/saas-product-analytics"

folders = [
    "data",
    "analysis",
    "dashboard"
]

for folder in folders:
    os.makedirs(os.path.join(base_path, folder), exist_ok=True)

print("Project structure created successfully!")


Project structure created successfully!


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [4]:
import pandas as pd
import numpy as np
from datetime import timedelta
import os

# -----------------------------
# CONFIG
# -----------------------------
np.random.seed(42)

BASE_PATH = "/content/drive/MyDrive/saas-product-analytics/data"
NUM_USERS = 1000
START_DATE = pd.to_datetime("2024-01-01")
END_DATE = pd.to_datetime("2024-06-30")

# -----------------------------
# USERS TABLE
# -----------------------------
user_ids = np.arange(1, NUM_USERS + 1)

signup_dates = pd.to_datetime(
    np.random.choice(
        pd.date_range(START_DATE, END_DATE),
        size=NUM_USERS
    )
)

countries = np.random.choice(
    ["India", "USA", "Canada", "UK", "Germany"],
    size=NUM_USERS,
    p=[0.35, 0.25, 0.15, 0.15, 0.10]
)

plan_types = np.random.choice(
    ["Free", "Standard", "Premium"],
    size=NUM_USERS,
    p=[0.6, 0.25, 0.15]
)

users_df = pd.DataFrame({
    "user_id": user_ids,
    "signup_date": signup_dates,
    "country": countries,
    "plan_type": plan_types
})

# -----------------------------
# SUBSCRIPTIONS TABLE
# -----------------------------
churn_flags = np.random.choice([0, 1], size=NUM_USERS, p=[0.65, 0.35])

end_dates = []
for i in range(NUM_USERS):
    if churn_flags[i] == 1:
        churn_days = np.random.randint(7, 90)
        end_dates.append(users_df.loc[i, "signup_date"] + timedelta(days=churn_days))
    else:
        end_dates.append(pd.NaT)

subscriptions_df = pd.DataFrame({
    "user_id": user_ids,
    "start_date": users_df["signup_date"],
    "end_date": end_dates
})

# -----------------------------
# EVENTS TABLE
# -----------------------------
events = []

event_types = ["login", "create_ticket", "comment", "board_view"]

for i in range(NUM_USERS):
    user_id = user_ids[i]
    signup_date = users_df.loc[i, "signup_date"]
    churned = churn_flags[i]

    if churned:
        active_days = np.random.randint(7, 60)
    else:
        active_days = np.random.randint(60, 150)

    num_events = np.random.randint(10, 80)

    for _ in range(num_events):
        event_day = signup_date + timedelta(days=np.random.randint(0, active_days))

        if event_day > END_DATE:
            continue

        event = np.random.choice(
            event_types,
            p=[0.45, 0.2, 0.2, 0.15]
        )

        events.append([user_id, event_day, event])

events_df = pd.DataFrame(
    events,
    columns=["user_id", "event_date", "event_type"]
)

# -----------------------------
# SAVE FILES
# -----------------------------
os.makedirs(BASE_PATH, exist_ok=True)
users_df.to_csv(f"{BASE_PATH}/users.csv", index=False)
subscriptions_df.to_csv(f"{BASE_PATH}/subscriptions.csv", index=False)
events_df.to_csv(f"{BASE_PATH}/events.csv", index=False)

print("Data generation complete!")
print(f"Users: {len(users_df)}")
print(f"Events: {len(events_df)}")

Data generation complete!
Users: 1000
Events: 35482


In [5]:
!ls /content/drive/MyDrive/saas-product-analytics/data

events.csv  subscriptions.csv  users.csv


In [6]:
import pandas as pd

base_path = "/content/drive/MyDrive/saas-product-analytics/data"

users = pd.read_csv(f"{base_path}/users.csv", parse_dates=["signup_date"])
events = pd.read_csv(f"{base_path}/events.csv", parse_dates=["event_date"])
subs = pd.read_csv(f"{base_path}/subscriptions.csv", parse_dates=["start_date", "end_date"])

users.head(), events.head()


(   user_id signup_date  country plan_type
 0        1  2024-04-12   Canada  Standard
 1        2  2024-06-28      USA      Free
 2        3  2024-04-02  Germany      Free
 3        4  2024-01-15  Germany      Free
 4        5  2024-04-16    India      Free,
    user_id event_date  event_type
 0        1 2024-04-29       login
 1        1 2024-05-11       login
 2        1 2024-05-15       login
 3        1 2024-06-12  board_view
 4        1 2024-05-03       login)

DAU — Daily Active Users

In [7]:
dau = (
    events
    .groupby(events["event_date"].dt.date)["user_id"]
    .nunique()
    .reset_index(name="DAU")
)

dau.head()


,event_date,DAU
0,2024-01-01,4
1,2024-01-02,6
2,2024-01-03,11
3,2024-01-04,12
4,2024-01-05,14


WAU — Weekly Active Users

In [8]:
events["week"] = events["event_date"].dt.to_period("W").astype(str)

wau = (
    events
    .groupby("week")["user_id"]
    .nunique()
    .reset_index(name="WAU")
)

wau.head()


,week,WAU
0,2024-01-01/2024-01-07,36
1,2024-01-08/2024-01-14,75
2,2024-01-15/2024-01-21,106
3,2024-01-22/2024-01-28,136
4,2024-01-29/2024-02-04,171


MAU — Monthly Active Users

In [9]:
events["month"] = events["event_date"].dt.to_period("M").astype(str)

mau = (
    events
    .groupby("month")["user_id"]
    .nunique()
    .reset_index(name="MAU")
)

mau.head()


,month,MAU
0,2024-01,160
1,2024-02,306
2,2024-03,411
3,2024-04,501
4,2024-05,573


Stickiness Ratio

In [10]:
avg_dau = dau["DAU"].mean()
avg_mau = mau["MAU"].mean()

stickiness = avg_dau / avg_mau
stickiness


np.float64(0.2996079384858524)

# Funnel Analysis

In [11]:
signup_users = set(users["user_id"])


In [12]:
login_users = set(
    events[events["event_type"] == "login"]["user_id"].unique()
)


In [13]:
ticket_users = set(
    events[events["event_type"] == "create_ticket"]["user_id"].unique()
)


In [14]:
comment_users = set(
    events[events["event_type"] == "comment"]["user_id"].unique()
)


In [15]:
funnel = pd.DataFrame({
    "stage": ["Signup", "Login", "Create Ticket", "Comment"],
    "users": [
        len(signup_users),
        len(login_users),
        len(ticket_users),
        len(comment_users)
    ]
})

funnel


,stage,users
0,Signup,1000
1,Login,974
2,Create Ticket,940
3,Comment,944


In [16]:
funnel["conversion_rate"] = funnel["users"].pct_change()
funnel


,stage,users,conversion_rate
0,Signup,1000,NaN
1,Login,974,-0.026000
2,Create Ticket,940,-0.034908
3,Comment,944,0.004255


*Funnel analysis shows where users drop off as they move from onboarding to deeper engagement. In our case, the largest drop occurs between login and ticket creation, indicating an activation problem rather than an acquisition problem.*

In [17]:
events_7d = events.merge(users, on="user_id")
events_7d["days_since_signup"] = (
    events_7d["event_date"] - events_7d["signup_date"]
).dt.days

events_7d = events_7d[events_7d["days_since_signup"] <= 7]

early_ticket_users = set(
    events_7d[events_7d["event_type"] == "create_ticket"]["user_id"]
)

len(early_ticket_users)


582

# RETENTION & COHORT ANALYSIS

In [18]:
import pandas as pd

base_path = "/content/drive/MyDrive/saas-product-analytics/data"

users = pd.read_csv(f"{base_path}/users.csv", parse_dates=["signup_date"])
events = pd.read_csv(f"{base_path}/events.csv", parse_dates=["event_date"])


In [19]:
users["cohort_month"] = users["signup_date"].dt.to_period("M")
users.head()


,user_id,signup_date,country,plan_type,cohort_month
0,1,2024-04-12,Canada,Standard,2024-04
1,2,2024-06-28,USA,Free,2024-06
2,3,2024-04-02,Germany,Free,2024-04
3,4,2024-01-15,Germany,Free,2024-01
4,5,2024-04-16,India,Free,2024-04


In [20]:
events_users = events.merge(users, on="user_id", how="left")
events_users.head()


,user_id,event_date,event_type,signup_date,country,plan_type,cohort_month
0,1,2024-04-29,login,2024-04-12,Canada,Standard,2024-04
1,1,2024-05-11,login,2024-04-12,Canada,Standard,2024-04
2,1,2024-05-15,login,2024-04-12,Canada,Standard,2024-04
3,1,2024-06-12,board_view,2024-04-12,Canada,Standard,2024-04
4,1,2024-05-03,login,2024-04-12,Canada,Standard,2024-04


In [21]:
events_users["event_month"] = events_users["event_date"].dt.to_period("M")

events_users["cohort_index"] = (
    events_users["event_month"] - events_users["cohort_month"]
).apply(lambda x: x.n)


In [22]:
cohort_data = (
    events_users
    .groupby(["cohort_month", "cohort_index"])["user_id"]
    .nunique()
    .reset_index()
)
cohort_data.head()


,cohort_month,cohort_index,user_id
0,2024-01,0,160
1,2024-01,1,156
2,2024-01,2,124
3,2024-01,3,82
4,2024-01,4,45


In [23]:
cohort_pivot = cohort_data.pivot(
    index="cohort_month",
    columns="cohort_index",
    values="user_id"
)
cohort_pivot


cohort_index,0,1,2,3,4,5
cohort_month,,,,,,
2024-01,160.0,156.0,124.0,82.0,45.0,8.0
2024-02,150.0,150.0,114.0,81.0,43.0,NaN
2024-03,137.0,136.0,103.0,76.0,NaN,NaN
2024-04,169.0,171.0,135.0,NaN,NaN,NaN
2024-05,173.0,181.0,NaN,NaN,NaN,NaN
2024-06,137.0,NaN,NaN,NaN,NaN,NaN


Monthly retention

In [24]:
cohort_size = cohort_pivot[0]
retention = cohort_pivot.divide(cohort_size, axis=0)
retention


cohort_index,0,1,2,3,4,5
cohort_month,,,,,,
2024-01,1.0,0.975000,0.775000,0.512500,0.281250,0.05
2024-02,1.0,1.000000,0.760000,0.540000,0.286667,NaN
2024-03,1.0,0.992701,0.751825,0.554745,NaN,NaN
2024-04,1.0,1.011834,0.798817,NaN,NaN,NaN
2024-05,1.0,1.046243,NaN,NaN,NaN,NaN
2024-06,1.0,NaN,NaN,NaN,NaN,NaN


Weekly retention

In [25]:
events_users["days_since_signup"] = (
    events_users["event_date"] - events_users["signup_date"]
).dt.days

events_users["week_index"] = events_users["days_since_signup"] // 7


In [26]:
weekly_cohort = (
    events_users
    .groupby(["cohort_month", "week_index"])["user_id"]
    .nunique()
    .reset_index()
)

weekly_pivot = weekly_cohort.pivot(
    index="cohort_month",
    columns="week_index",
    values="user_id"
)

weekly_retention = weekly_pivot.divide(weekly_pivot[0], axis=0)
weekly_retention


week_index,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
cohort_month,,,,,,,,,,,,,,,,,,,,,
2024-01,1.0,0.987500,0.900000,0.912500,0.806250,0.775000,0.737500,0.693750,0.606250,0.575000,...,0.431250,0.418750,0.343750,0.318750,0.243750,0.250000,0.137500,0.100000,0.081250,0.025000
2024-02,1.0,0.993464,0.908497,0.862745,0.797386,0.790850,0.732026,0.660131,0.601307,0.549020,...,0.470588,0.444444,0.352941,0.326797,0.261438,0.189542,0.163399,0.124183,0.045752,0.039216
2024-03,1.0,0.971631,0.893617,0.872340,0.773050,0.787234,0.716312,0.631206,0.617021,0.574468,...,0.531915,0.468085,0.347518,0.241135,0.170213,0.078014,0.028369,NaN,NaN,NaN
2024-04,1.0,0.982143,0.970238,0.928571,0.886905,0.845238,0.797619,0.714286,0.696429,0.583333,...,0.214286,0.083333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05,1.0,0.988506,0.971264,0.862069,0.787356,0.637931,0.396552,0.235632,0.068966,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06,1.0,0.738462,0.530769,0.269231,0.015385,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
from google.colab import auth
auth.authenticate_user()


In [28]:
users.to_csv("/content/users.csv", index=False)
events.to_csv("/content/events.csv", index=False)


In [29]:
import pandas as pd

users["signup_date"] = pd.to_datetime(users["signup_date"])
events["event_date"] = pd.to_datetime(events["event_date"])


In [30]:
users["cohort_month"] = users["signup_date"].dt.to_period("M")


In [31]:
events_users = events.merge(users, on="user_id", how="left")


In [32]:
events_users["event_month"] = events_users["event_date"].dt.to_period("M")

events_users["month_index"] = (
    events_users["event_month"] - events_users["cohort_month"]
).apply(lambda x: x.n)


In [33]:
cohort_counts = (
    events_users
    .groupby(["cohort_month", "month_index"])["user_id"]
    .nunique()
    .reset_index(name="active_users")
)


In [34]:
cohort_sizes = (
    users
    .groupby("cohort_month")["user_id"]
    .nunique()
    .reset_index(name="cohort_size")
)


In [35]:
cohort_retention = cohort_counts.merge(
    cohort_sizes,
    on="cohort_month"
)

cohort_retention["retention_rate"] = (
    cohort_retention["active_users"] / cohort_retention["cohort_size"]
)


In [36]:
cohort_retention = cohort_retention[
    ["cohort_month", "month_index", "retention_rate"]
].sort_values(["cohort_month", "month_index"])


In [37]:
cohort_retention["cohort_month"] = cohort_retention["cohort_month"].astype(str)


In [38]:
cohort_retention.to_csv("cohort_retention.csv", index=False)
